In [1]:
import time, datetime
import requests 
import pickle as pkl 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import json

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn import metrics
import time
from sklearn.model_selection import ShuffleSplit

import itertools

import warnings
warnings.filterwarnings('ignore')


from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.model_selection import train_test_split

import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from scipy import stats
import pprint
import seaborn as sns

#models 
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.ensemble import VotingClassifier  

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from collections import Counter

import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import StratifiedShuffleSplit

from vecstack import stacking
from xgboost import XGBClassifier


import Framework


In [2]:
'''
Step 1
preprocess data
'''

# Prepare data
# data = pd.read_csv("mts_june_10m_rh.csv", dtype='float').drop(['Unnamed: 0'], axis=1)

# X = data.drop(['cs'], axis=1)
# Y = data['cs']

data = pd.read_pickle(r'all_metrics_data_3h.pkl')
# data = pd.read_pickle(r'all_metrics_data_12h.pkl')

X = data.drop(['time', 'label', "txn_fail_num"], axis=1)
Y = data.label

# Preproces for PCA
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(X)

# Create PCA
pca = PCA(n_components = 0.95)
pca_df = pca.fit_transform(scaled_df)
pca.fit(X)

PCA(n_components=0.95)

In [3]:
'''
Step 2
Define models
'''

dbscan_model = DBSCAN(eps=3, min_samples=2)
isolation_forest_model = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
kmeans_model = KMeans(n_clusters=2)
knn_model = KNeighborsClassifier()
svm_model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
rf_model = RandomForestClassifier(max_depth=10, random_state=5)

all_models = [dbscan_model, isolation_forest_model, kmeans_model, knn_model, svm_model, rf_model]
unsupervised_models = [dbscan_model, isolation_forest_model, kmeans_model]
supervised_models = [knn_model, svm_model, rf_model]

In [4]:
'''
Step 2
Build the framework
'''
fmsad = Framework.FMSAD(X,Y, supervised_models, unsupervised_models)

fmsad.correlation_analysis()

selected_feature_names = fmsad.correlation_features()

In [5]:
'''
Step 3
Initiate data set variations
This can be used to create other models later
'''

all_feature_X = X
correlation_analysis_X = X[selected_feature_names]
PCA_X = pca_df.copy()

In [ ]:
'''
Step 4
Fit weights for weighted model
parameters: models for ensemble to optimize,
            Weight range to optimize
'''

# Optimize Unsupervised model weights
fmsad.optimize_voting_parameters(list(itertools.permutations([1,1,1]))) # Change weights to desired range



# Optimize All model weights
fmsad.optimize_all_voting_parameters(all_models, list(itertools.permutations([1,1,1,1,1,1]))) # Change weights to desired range



In [7]:
'''
Step 5
Build the framework
'''
# Run framework on selected models
fmsad.generate_models([[0,3,2],[0,3,2],[3,1,1,3,2,2]])

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [IsolationForest]
    fold  0:  [0.36697248]
    fold  1:  [0.30275229]
    fold  2:  [0.38532110]
    fold  3:  [0.35779817]
    ----
    MEAN:     [0.35321101] + [0.03077158]
    FULL:     [0.35321101]

model  1:     [KMeans]
    fold  0:  [0.57798165]
    fold  1:  [0.39449541]
    fold  2:  [0.49541284]
    fold  3:  [0.41284404]
    ----
    MEAN:     [0.47018349] + [0.07292712]
    FULL:     [0.47018349]

[10:35:02] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.94495413]

,model,accuracy,precision,recall,f1 score,time
7,supervised stacking,0.979452,0.975410,0.982955,0.978724,0.990242
8,all model stacking,0.972603,0.967742,0.977273,0.971705,2.690768
4,"(StandardScaler(), SVC(gamma='auto'))",0.966438,0.964713,0.968236,0.966078,0.036561
13,supervised weighted voting,0.966438,0.964713,0.968236,0.966078,0.227069
5,"(DecisionTreeClassifier(max_depth=10, max_feat...",0.963699,0.962400,0.965111,0.963284,0.173914
10,supervised voting,0.963014,0.961252,0.965358,0.962652,0.293000
14,all model weighted voting,0.956164,0.955284,0.956002,0.955542,0.865067
3,KNeighborsClassifier(),0.943151,0.941690,0.944074,0.942489,0.042231
11,all model voting,0.708219,0.743639,0.678506,0.673475,0.657184
6,unsupervised stacking,0.678082,0.788606,0.597766,0.562679,2.225617
